# 6 - Transformers for Sentiment Analysis

Source: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb

In [0]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
!pip install transformers

     |████████████████████████████████| 573kB 6.5MB/s 
     |████████████████████████████████| 870kB 20.5MB/s 
     |████████████████████████████████| 1.0MB 38.0MB/s 
     |████████████████████████████████| 3.7MB 41.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.40-cp36-none-any.whl size=893351 sha256=97df08980244eb8b8439ff2955981ddbf4d02d810fadf759a473ff956fe1fdbe
  Stored in directory: /root/.cache/pip/wheels/ba/44/6e/939fb8625572dca96344566c5167bc4f8ab079294509898141
Successfully built sacremoses


In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
len(tokenizer.vocab)

30522

In [8]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [9]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [10]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [11]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [12]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [13]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [0]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [0]:

from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [16]:

from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.7MB/s]


In [17]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [18]:
print(vars(train_data.examples[6]))

{'text': [1996, 3819, 4028, 2003, 17910, 2098, 2043, 1037, 2564, 1006, 2209, 2011, 2984, 9155, 3345, 2953, 1010, 2320, 1996, 2564, 2000, 2472, 2728, 27838, 4168, 18009, 2015, 1010, 2040, 16254, 2098, 2023, 2792, 1007, 1010, 2040, 9916, 2014, 3129, 2007, 1037, 11662, 1010, 2038, 1996, 28616, 13028, 9816, 1997, 4909, 1037, 10367, 2004, 2016, 2003, 2055, 2000, 2693, 1996, 2303, 2648, 1012, 1012, 2019, 6376, 9577, 28441, 5102, 1999, 1037, 4203, 19118, 4848, 1006, 2209, 2011, 1037, 14386, 13453, 22293, 6554, 7867, 1007, 1012, 2016, 21713, 5104, 2005, 2014, 2166, 2096, 2667, 2000, 2424, 1037, 2126, 1997, 6318, 2014, 3129, 1005, 1055, 11547, 1012, 2016, 7288, 2000, 2224, 2019, 22260, 1010, 2320, 2016, 12482, 1996, 4203, 6359, 2040, 22182, 2195, 9592, 2000, 24494, 2125, 1996, 2450, 1005, 1055, 2132, 1010, 2000, 4853, 1996, 6359, 2005, 2014, 3129, 1005, 1055, 4028, 1012, 4203, 6359, 11223, 2014, 1999, 1037, 9346, 1998, 7323, 2015, 1996, 2450, 1005, 1055, 2684, 2004, 2016, 5363, 7143, 2000, 2489

In [19]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['the', 'perfect', 'murder', 'is', 'foil', '##ed', 'when', 'a', 'wife', '(', 'played', 'by', 'mary', 'ellen', 'train', '##or', ',', 'once', 'the', 'wife', 'to', 'director', 'robert', 'ze', '##me', '##cki', '##s', ',', 'who', 'helm', '##ed', 'this', 'episode', ')', ',', 'who', 'murders', 'her', 'husband', 'with', 'a', 'poker', ',', 'has', 'the', 'mis', '##fort', '##une', 'of', 'receiving', 'a', 'visitor', 'as', 'she', 'is', 'about', 'to', 'move', 'the', 'body', 'outside', '.', '.', 'an', 'escaped', 'insane', 'madman', 'dressed', 'in', 'a', 'santa', 'claus', 'suit', '(', 'played', 'by', 'a', 'devi', '##ously', 'hideous', 'larry', 'drake', ')', '.', 'she', 'fen', '##ds', 'for', 'her', 'life', 'while', 'trying', 'to', 'find', 'a', 'way', 'of', 'hiding', 'her', 'husband', "'", 's', 'corpse', '.', 'she', 'decides', 'to', 'use', 'an', 'ax', ',', 'once', 'she', 'downs', 'the', 'santa', 'killer', 'who', 'misses', 'several', 'chances', 'to', 'chop', 'off', 'the', 'woman', "'", 's', 'head', ',', 

In [0]:
LABEL.build_vocab(train_data)

In [21]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f1158436ae8>, {'neg': 0, 'pos': 1})


In [0]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [23]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [0]:

import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [0]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [26]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [0]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [28]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [29]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 6m 47s
	Train Loss: 0.510 | Train Acc: 73.75%
	 Val. Loss: 0.274 |  Val. Acc: 88.86%
Epoch: 02 | Epoch Time: 6m 47s
	Train Loss: 0.274 | Train Acc: 88.81%
	 Val. Loss: 0.245 |  Val. Acc: 90.33%
Epoch: 03 | Epoch Time: 6m 47s
	Train Loss: 0.236 | Train Acc: 90.65%
	 Val. Loss: 0.234 |  Val. Acc: 90.88%
Epoch: 04 | Epoch Time: 6m 47s
	Train Loss: 0.207 | Train Acc: 91.92%
	 Val. Loss: 0.224 |  Val. Acc: 91.44%
Epoch: 05 | Epoch Time: 6m 47s
	Train Loss: 0.178 | Train Acc: 93.14%
	 Val. Loss: 0.235 |  Val. Acc: 91.15%


In [39]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.213 | Test Acc: 91.88%


In [0]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [41]:
predict_sentiment(model, tokenizer, "This film is terrible")

0.013714319095015526

In [42]:
predict_sentiment(model, tokenizer, "This film is great")

0.9431144595146179